In [3]:
# proof of concept/testing different structures of the wrapper.
# going to refactor the entire structure today so that its easier to scale it / more intuitive to use.

In [5]:
# api functions.
from datetime import datetime
from pandas.core.groupby import DataError as DataError
from alpha_vantage.foreignexchange import ForeignExchange
from yahoofinancials import YahooFinancials
import pandas as pd
class HiddenAlphabetAPI(object):
    
    
    def __init__(self, ticker, pandas=False):
        
        """
        this will call our raw time series API. 
        
        # so a developer says HiddenAlphabetAPI('AAPL', '1/1/2019', '1/5/2019')
        # and it returns a data time series object that they can then interact with.
        """
        
        self.ticker = ticker.upper()
        
        # as of now both start and enddates are pandas timestamped times.
        #self.start = pd.to_datetime(start)
        #self.end = pd.to_datetime(end)
        
        # where it says time_series is where we send it to your api.
        # i think we should be returning a timestamp and a sentiment score
        # all manipulation of the data will be done here.
        
        self.raw_data =  pd.read_csv('trading-tweets.csv')[['time', 'text']].to_dict()

        self.time_series_data = pd.DataFrame(self.raw_data).to_dict()
        self.aggregated_series = None
        self.pandas = pandas
        #self.min_date = self.handle_dates()['min_date']
        #self.max_date = self.handle_dates()['max_date']
    
    """def handle_dates(self):
        
        # helper to easily update self.min_date, self.max_date so we know our window.
        
        try:
            df = pd.Dataframe(self.time_series_data)
            df['datetime'] = pd.to_datetime(df.time)
            df = df.set_index('datetime')
            
            return {'min_date': min(df.index), 'max_date': max(df.index)}
        except:
            raise('ooops')"""
            
    
        
    def trim_time_series():
        
        """
        takes the time series and cuts excess days from both sides.
        """
        pass
    
    
    def pandas_check_return(self):
        if self.pandas:
            return pd.DataFrame(self.time_series_data)
        else:
            return self.time_series_data
        
    def get_sentiment(self):
        
        # dictionary/pandas dataframe where keys/columns names are time and text
        df = pd.DataFrame(self.time_series_data)

        # our sentiment model
        model = Sentiment()

        def vectorized_sentiment(text):
            try:
                return model.score(text)
            except:
                return {'score': 0, 'confidence_interval': (0, 0)}
        df['score_verbose'] = df.text.apply(vectorized_sentiment)
        
        df['sentiment_score'] = df.score_verbose.apply(lambda x: x['score'])
        df['lower_bound'] = df.score_verbose.apply(lambda x: x['confidence_interval'][0])
        df['upper_bound'] = df.score_verbose.apply(lambda x: x['confidence_interval'][1])
        self.time_series_data = df.to_dict()
        
        return self.pandas_check_return()
    
    def get_stock_price_data():
        """
        -> Calls our internal API
        -> Checks if we have the data.
        -> If so we return it.
        -> If not we go and get it, store it, then return it.
        """
        pass
    
    def get_currency_data():
        

        #ALPHAVANTAGE_API_KEY = 'KT3PLVIJ5XIWZLDQ'

        #fx = ForeignExchange(key = ALPHAVANTAGE_API_KEY)
        #usd = fx.get_currency_exchange_daily(from_symbol='BTC', to_symbol='USD')
        pass
    
    def remove_null_scores(self, verbose=True):
        
        """
        removes nulls - should check if the time series has already had the nulls removed.
        could do this by setting self.nulls_removed = False until self.remove_null_scores() gets called. 
        then self.nulls_removed = True, when nulls_removed gets called store pre_removed_mean_sentiment maybe?
        """
        
        df = pd.DataFrame(self.time_series_data)
        
        if verbose:
            self.time_series_data = df[df.sentiment_score != 0].to_dict()
            print('Average Sentiment Score Before: ' + str(round(df.sentiment_score.mean(), 3)) + '\n'
                       'Average Sentiment Score After: ' + str(round(pd.DataFrame(self.time_series_data).sentiment_score.mean(), 3)))
            return self.pandas_check_return()
        else:
            self.time_series_data = df[df.sentiment_score != 0].to_dict()
            return self.pandas_check_return()
    
    def aggregate_data(self, aggregate_by='day'):
        
        try:
            if aggregate_by == 'day':
                df = pd.DataFrame(self.time_series_data)
                df = df.groupby(pd.to_datetime(df.time).dt.date).mean()
                self.time_series_data = df.to_dict()
                
                return self.pandas_check_return()
            else:
                print('we dont support that aggregation yet, comment on our repo www.github.com/HiddenAlphabet')
        except DataError as e:
            raise e
        
        #print('No numeric types to aggregate. Have you added sentiment scores to the time series?')
    
    
    def sentiment_volatility(self, periods, min_periods=1):
        """
        gets the volatility of the sentiment score. 
        
        the period over which the vol is calculated can be a single period or a list of periods.
        """
        
        df = self.to_pandas()
        
        df['rolling_mean_sentiment'] = df.sentiment_score.rolling(window=periods, min_periods=1).mean()
        df['5day_vol'] = (((df['rolling_mean_sentiment'] - df.sentiment_score) ** 2)
                          .rolling(window=periods, min_periods=1).sum() / periods)**(1/2)
        self.time_series_data = df.drop(columns=['rolling_mean_sentiment']).to_dict()
        return self.pandas_check_return()
    
    
    def to_pandas(self, set_global=False):
        
        if set_global:
            self.pandas = True
        
        return pd.DataFrame(self.time_series_data)
    
    
    
    def get_vix_data(self, start, end, interval='daily'):
        
        vix = ['^VIX']
        vix_price = YahooFinancials(vix).get_historical_price_data('2019-06-03', '2019-06-13', 'daily')
        name_maps = {}

        for col in pd.DataFrame(vix_price['^VIX']['prices']).columns:
            name_maps[col] = 'vix_' + col 

        df = pd.DataFrame(vix_price['^VIX']['prices']).rename(columns=name_maps)
        df = df.groupby(pd.to_datetime(df.vix_formatted_date).dt.date).mean()
        
        #self.minimum_date = min(self.minimum_date, min(df.time)) or min(df.time)
        #self.maximum_date = max(self.minimum_date, max(df.time)) or max(df.time)
        
        idx = pd.period_range(min(df.vix_time), max(df.vix_time))
        df = df.reindex(idx, fill_value=0)
        
        main_ts_df = self.to_pandas()
        main_ts_df.reindex(idx, fill_value=0)
        for col in df.columns:
            main_ts_df[col] = df[col].values
            
        self.time_series_data = main_ts_df.to_dict()
        return self.pandas_check_return()

In [7]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
lemmatizer = WordNetLemmatizer()
# this whole cell will be automated when pip installed using os library. will modify when deploying to Pypi

import ast
import numpy as np

def create_ci_lexicon(lexicon_file):
    
    """
    Takes in an inital lexicon with information on how the final values was derived.
    Outputs confidence score for each individual word.
    """
    
    df = pd.read_csv(lexicon_file, encoding='cp437', header=None)
    
    def descriptive(x):
        return np.std(ast.literal_eval(x))
    
    df['std_dev'] = df[3].str.strip("'").apply(descriptive)
    
    basis = df.std_dev.mean() + 1.5 * df.std_dev.std()
    
    def normalize_conf(x):
        return (x - basis) / basis
    
    df['norm_conf'] = df.std_dev.apply(normalize_conf)
    
    return dict(zip(df[[0, 'norm_conf']][0].values, (1 - (df.norm_conf.rank() / df.norm_conf.rank().max()))))

ci_lexicon = create_ci_lexicon('vader_lexicon.csv')
lexicon = dict(zip(pd.read_csv('vader_lexicon.csv', encoding='cp437', header=None)[[0, 1]].values.T[0], pd.read_csv('vader_lexicon.csv', encoding='cp437', header=None)[[0, 1]].values.T[1]))

C:\Users\mac\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [8]:
#Here are a list of words that, when preceding, require us to inverse our score. 
#"I like Apple" has a score of 1.5, but "I don't like Apple" would have a score of -1.5 because of the negation
negate = \
    ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

boost = \
    ['absolutely', 'amazingly', 'awfully', 'completely', 'considerably', 'decidedly', 'deeply', 'effing', 'enormously',
     'entirely', 'especially', 'exceptionally', 'extremely', 'fabulously', 'flipping', 'flippin', 'fricking', 'frickin',
     'frigging', 'friggin', 'fully', 'fucking', 'greatly', 'hella', 'highly', 'hugely', 'incredibly', 'intensely', 'majorly',
     'more', 'most', 'particularly', 'purely', 'quite', 'really', 'remarkably', 'so', 'substantially',
     'thoroughly', 'totally', 'tremendously', 'uber', 'unbelievably', 'unusually', 'utterly', 'very', 'almost', 'barely']

In [10]:
class PreprocessText(object):
    """
    Sentiment relevant text properties. 
    """

    def __init__(self, text):

        self.text = text
        self.clean_text = self._clean_text()

    def _clean_text(self):

        wordz = self.text.split()

        lemmatizer = WordNetLemmatizer()
        words = []
        for word in wordz:
            words.append(lemmatizer.lemmatize(word))

        return words
    
    
class Sentiment(object):
    """
    Sentiment Analyzer
    """


    def __init__(self):

        self.lexicon = lexicon
        self.negate = negate
        self.ci_lexicon = ci_lexicon
            
    def score(self, text):

        words = PreprocessText(text).clean_text
        sentiments = []
        confidences = []
        for item in words:
            sentiments, confidences = self.sentiment_polarity(item, sentiments, confidences, words)
        score_interval = self.final_calculation(sentiments, confidences)
        
        return {'score': score_interval[0], 'confidence_interval': score_interval[1]}
        
    def sentiment_polarity(self, item, sentiments, confidences, words):
        #Checks the average sentiment score by querying our lexicon
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            weight = self.lexicon[item_lowercase] / 5 
            final_weight = self.negation_check(words, item, weight)
            confidence_score = self.ci_lexicon[item_lowercase]
            confidence_calc = self.confidence_calc(final_weight, confidence_score)
            sentiments.append(final_weight)
            confidences.append(confidence_calc)
        return sentiments, confidences
    
    
    def final_calculation(self, sentiments, confidences):
        #Applies all final calculations to get the final sentiment score
        
        if len(sentiments) == 0:
            final_calc = 0
            return final_calc, (0, 0)
        else:
            final_calc = sum(sentiments) / len(sentiments)
        
        count = 0
        upper = 0
        lower = 0
        for interv in confidences:
            upper += interv[1]
            lower += interv[0]
            count += 1
        
        return final_calc, (lower/count, upper/count)
    
    def negation_check(self, words, item, weight):
        #Checks to see if there is a negation word in the words
        #"I love apples = 3.2"
        #"I don't love apples = -3.2"
        neg_coef = 1
        lexicon_index = words.index(item)
        preceding_word = words[lexicon_index - 1]
        if preceding_word in self.negate:
            neg_coef = -1
        final_weight = weight * neg_coef
        return final_weight
    
    def confidence_calc(self, final_weight, confidence_score): 
        
        if final_weight >= 0:
            upper = final_weight + (1 - confidence_score) * final_weight
            lower = final_weight - (1 - confidence_score) * final_weight
        else: 
            lower = final_weight + (1 - confidence_score) * final_weight
            upper = final_weight - (1 - confidence_score) * final_weight
        return (lower, upper)
        
    

In [11]:
# should probably 

api = HiddenAlphabetAPI('AAPL', pandas=True)
api.get_sentiment()

,lower_bound,score_verbose,sentiment_score,text,time,upper_bound
0,-0.328855,"{'score': -0.24, 'confidence_interval': (-0.32...",-0.240000,"Hoy nos toca irnos en negativo 1,25. Un mercad...",6/13/2019 17:04,-0.151145
1,0.004554,"{'score': 0.06, 'confidence_interval': (0.0045...",0.060000,🏄 #GBPUSD Head &amp; Shoulders Pattern Challe...,6/13/2019 17:04,0.115446
2,0.116533,"{'score': 0.27999999999999997, 'confidence_int...",0.280000,$SPX #stock $AVGO highlights the very short li...,6/13/2019 17:03,0.443467
3,0.000000,"{'score': 0, 'confidence_interval': (0, 0)}",0.000000,"As we always said, there are traders and then ...",6/13/2019 17:03,0.000000
4,0.234261,"{'score': 0.54, 'confidence_interval': (0.2342...",0.540000,RT @EarnsBit: Hello Community!\r\nWe are happy...,6/13/2019 17:03,0.845739
5,0.000000,"{'score': 0, 'confidence_interval': (0, 0)}",0.000000,#neo #price #analysis: #neo/usd #bulls #outper...,6/13/2019 17:03,0.000000
6,0.000000,"{'score': 0, 'confidence_interval': (0, 0)}",0.000000,@eToro Milioni di illusi hanno pensato che l'a...,6/13/2019 17:02,0.000000
7,-0.358442,"{'score': -0.07999999999999999, 'confidence_in...",-0.080000,Failure defeats losers but inspires\r\nwinners...,6/13/2019 17:02,0.198442
8,-0.337355,"{'score': -0.18000000000000002, 'confidence_in...",-0.180000,Blue Boxes will give you 85% chance to get a b...,6/13/2019 17:02,-0.022645
9,0.000000,"{'score': 0, 'confidence_interval': (0, 0)}",0.000000,Buying ETH at current price: $259.42 #ethusd #...,6/13/2019 17:02,0.000000


In [ ]:
# 